In [8]:
pip install keyphrase-vectorizers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install keybert


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from keyphrase_vectorizers import KeyphraseCountVectorizer



from keybert import KeyBERT
kw_model = KeyBERT()
from transformers import pipeline

In [12]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
from google.colab import files
uploaded = files.upload()



Saving new_data (3).csv to new_data (3) (3).csv


In [41]:
import io
data = pd.read_csv(io.BytesIO(uploaded['new_data (3).csv']))


In [49]:
data.head()


,topic,keyword,short_text
0,Information Technology,NaN,Knowing instructors have experiential knowledg...
1,Information Technology,NaN,This module consists of a PowerPoint presentat...
2,Information Technology,NaN,Geo-information has proven to be extremely he...
3,Information Technology,NaN,Engineering and Technology for the Better Good...
4,Information Technology,NaN,Mushroom cultivation technology course provide...


In [37]:
data['topic'].unique().tolist()

['Information Technology',
 'Sciences',
 'Humanities',
 'Mathematics',
 'Social Sciences',
 'Sustainability',
 'Other',
 nan,
 'Robotics',
 'Geography and History',
 'Biography']

In [57]:
data=data.query('topic=="Information Technology"')

data=data.reset_index(drop=True)
data.duplicated()


0

In [18]:
def extractor(text):
    return kw_model.extract_keywords(text, vectorizer=KeyphraseCountVectorizer(),top_n=1, stop_words=None)
     

In [19]:
data['keyword']=data['short_text'].apply(extractor)
data['keyword']


2022-11-16 18:50:09,435 - KeyphraseVectorizer - INFO - It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
INFO:KeyphraseVectorizer:It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0      [(technology integration professional developm...
1                                [(hev systems, 0.5731)]
2             [(disaster management activities, 0.6009)]
3            [(engineering technology students, 0.5274)]
4      [(mushroom cultivation technology course, 0.72...
                             ...                        
294           [(information technology careers, 0.6398)]
295                            [(teleradiology, 0.5724)]
296                      [(bacteria subclasses, 0.4917)]
297                            [(atetv project, 0.7306)]
298                              [(ict podcast, 0.7098)]
Name: keyword, Length: 299, dtype: object

In [20]:
def justkey(text):
  return text[0][0]

In [21]:
data['keyword']=data['keyword'].apply(justkey)
data_new=data.copy()

In [22]:
corpus=np.array(data_new['short_text'])

In [23]:
corpus

array(['Knowing instructors have experiential knowledge of discipline-specific information and instruction success rates, this course is intended to convey an alternative to face-to-face instruction\xa0and aims to increase the commitment and desire of faculty to utilize online tools for instruction. Users\xa0I make no assumption about age or gender of learners; however, educational discipline and background are acknowledged, as well as the assumption that faculty have the knowledge to effectively operate a smart deviceSelf-Paced Study\xa0this course can be utilized as self-paced and focused on only those tools best suited according to discipline. Learning Objectives\xa0Learners will choose commonly utilized tools, technology and/or digital sources and identify technology(s) support of blended or fully online environment.Learners will use a technology evaluation rubric to evaluate a technology tool selected to meet a specific learning goal.Learners will develop a Technology Integration 

In [24]:
corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus} 
                         )
corpus_df = corpus_df[['Document']]
corpus_df

,Document
0,Knowing instructors have experiential knowledg...
1,This module consists of a PowerPoint presentat...
2,Geo-information has proven to be extremely he...
3,Engineering and Technology for the Better Good...
4,Mushroom cultivation technology course provide...
...,...
294,This paper examines the role of lower income w...
295,"Dr Sanjoy Sanyal, then Associate Professor, pr..."
296,This video was recorded at 15th ACM SIGKDD Int...
297,The ATETV project delivers web-based videos to...


In [25]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
import nltk
import re

In [27]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, flags=re.I|re.A) # [^a-zA-Z\s] => remove any digits, special characters, symbols etc.
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

In [28]:
normalize_corpus = np.vectorize(normalize_document) # this is same as running the function for all documents in a for loop

norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['knowing instructors experiential knowledge disciplinespecific information instruction success rates course intended convey alternative facetoface instructionand aims increase commitment desire faculty utilize online tools instruction usersi make assumption age gender learners however educational discipline background acknowledged well assumption faculty knowledge effectively operate smart deviceselfpaced studythis course utilized selfpaced focused tools best suited according discipline learning objectiveslearners choose commonly utilized tools technology andor digital sources identify technologys support blended fully online environmentlearners use technology evaluation rubric evaluate technology tool selected meet specific learning goallearners develop technology integration professional development plan include areas need desire use',
       'module consists powerpoint presentation labs syllabus designed enhance automotive electrical courses hev technologiesand developed seed

In [29]:
tv = TfidfVectorizer(min_df=0., max_df=1., norm='l2', use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names_out()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,aa,aac,aace,aacsb,aamodifications,aba,abalone,abbasid,abbreviations,abc,...,younger,youre,youth,youtube,zeit,zines,zip,zooming,zoos,zygote
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
similarity_matrix = cosine_similarity(tv_matrix)
similarity_df = pd.DataFrame(similarity_matrix)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,289,290,291,292,293,294,295,296,297,298
0,1.000000,0.015222,0.048853,0.029419,0.018021,0.009734,0.057300,0.016461,0.027389,0.018252,...,0.023179,0.024186,0.026381,0.026410,0.048138,0.005704,0.021392,0.028316,0.015116,0.005986
1,0.015222,1.000000,0.011506,0.022775,0.003887,0.039968,0.062729,0.012424,0.077996,0.181390,...,0.020686,0.009232,0.001214,0.026240,0.028559,0.013155,0.003899,0.000000,0.001030,0.004757
2,0.048853,0.011506,1.000000,0.023588,0.012944,0.021206,0.028552,0.008442,0.038355,0.001225,...,0.011746,0.005525,0.002357,0.019462,0.047913,0.001359,0.011540,0.020585,0.014621,0.007157
3,0.029419,0.022775,0.023588,1.000000,0.017677,0.016869,0.021707,0.014109,0.038932,0.013420,...,0.004631,0.038681,0.009410,0.044482,0.043920,0.017392,0.032462,0.020554,0.026059,0.010936
4,0.018021,0.003887,0.012944,0.017677,1.000000,0.002409,0.021509,0.376156,0.013107,0.013762,...,0.004200,0.002656,0.021215,0.003459,0.016601,0.004732,0.023003,0.000922,0.017725,0.004966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,0.005704,0.013155,0.001359,0.017392,0.004732,0.001525,0.029625,0.004635,0.048247,0.013986,...,0.002658,0.023509,0.052094,0.002189,0.004431,1.000000,0.020624,0.000584,0.043440,0.029624
295,0.021392,0.003899,0.011540,0.032462,0.023003,0.005109,0.095004,0.020190,0.008150,0.007489,...,0.022214,0.017070,0.003467,0.007077,0.007288,0.020624,1.000000,0.004082,0.039186,0.007383
296,0.028316,0.000000,0.020585,0.020554,0.000922,0.012076,0.003042,0.001203,0.003595,0.000000,...,0.014688,0.018303,0.005382,0.017789,0.040201,0.000584,0.004082,1.000000,0.003794,0.003218
297,0.015116,0.001030,0.014621,0.026059,0.017725,0.004487,0.014737,0.011376,0.010794,0.006125,...,0.005873,0.006123,0.019755,0.006906,0.006971,0.043440,0.039186,0.003794,1.000000,0.006945


In [31]:
from sklearn.cluster import KMeans

In [32]:
km = KMeans(n_clusters=10, random_state=0)
km.fit_transform(similarity_matrix)
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])
total=pd.concat([corpus_df, cluster_labels,data_new['keyword']], axis=1)
total

,Document,ClusterLabel,keyword
0,Knowing instructors have experiential knowledg...,9,technology integration professional developmen...
1,This module consists of a PowerPoint presentat...,3,hev systems
2,Geo-information has proven to be extremely he...,8,disaster management activities
3,Engineering and Technology for the Better Good...,8,engineering technology students
4,Mushroom cultivation technology course provide...,3,mushroom cultivation technology course
...,...,...,...
294,This paper examines the role of lower income w...,3,information technology careers
295,"Dr Sanjoy Sanyal, then Associate Professor, pr...",3,teleradiology
296,This video was recorded at 15th ACM SIGKDD Int...,3,bacteria subclasses
297,The ATETV project delivers web-based videos to...,4,atetv project


In [33]:
cluster_labels.value_counts()

ClusterLabel
3               117
9                61
8                51
1                17
7                17
0                13
2                 9
4                 6
5                 4
6                 4
dtype: int64